1. **Descarga e Instalación de Librerías**

In [9]:
import pandas as pd
import numpy as np
import nltk
import unidecode
import unicodedata
import string
from num2words import num2words
from collections import Counter
from word2number import w2n
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import FreqDist

In [2]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.0 MB/s eta 0:00:00


In [3]:
!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32896 sha256=3356a5a3258cbeca141cd0aec2fc6e0da2c41859d6d430ee7487efe14f5a6929
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [4]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=5a760d6334e0bc3b31ac104dcc6d82e9e39bb5fc6d09e8809826e24871c40f14
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
!pip install word2number

  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5567 sha256=0a07199afc55c678817f05b387a518b408666302d7559db56650d8bb746a4ec3
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [6]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

2. **Carga de Dataset**

In [19]:
#Montar Drive

from google.colab import drive
drive.mount('/content/drive')

#Cargar archivo en DataFrame --> df_corpus

file_path = '//content/drive/MyDrive/Bootcamp AI&ML KC/NLP/Software.json.gz'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_corpus = pd.read_json(file_path, compression='gzip', lines=True)
df_corpus.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,4,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,1,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7,NaN
3,3,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3,NaN
4,5,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


3. **Limpieza y Preprocesamiento de Datos**

3.1  Seleccionar las Columnas de Interés (Eliminar Columnas Innecesarias)

In [20]:
#Columnas de Interés --> 'overall', 'reviewText'

columnas_a_conservar = ['overall', 'reviewText']
df_corpus = df_corpus[columnas_a_conservar]
df_corpus.head(5)

,overall,reviewText
0,4,The materials arrived early and were in excell...
1,4,I am really enjoying this book with the worksh...
2,1,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ..."
3,3,This book was missing pages!!! Important pages...
4,5,I have used LearnSmart and can officially say ...


3.2 Identificar NaN y Nulls en Columnas

In [21]:
# Validar NaN o nulos en las columnas "overall" y "reviewText"
nan_overall = df_corpus['overall'].isna().sum()
nan_reviewText = df_corpus['reviewText'].isna().sum()

# Mostrar la cantidad de NaN o nulos en cada columna
print(f"NaN o nulos en 'overall': {nan_overall}")
print(f"NaN o nulos en 'reviewText': {nan_reviewText}")

NaN o nulos en 'overall': 0
NaN o nulos en 'reviewText': 66


3.3 Eliminar o Imputar valores/datos faltantes (missing values)


In [22]:
#Eliminar NaN de reviewText
#La proporción de NaN es 0.01436% del total de reviews del df
df_corpus = df_corpus.dropna(subset=['reviewText'])
nan_reviewText = df_corpus['reviewText'].isna().sum()
print(f"NaN o nulos en 'reviewText': {nan_reviewText}")

NaN o nulos en 'reviewText': 0


3.4 Preprocesamiento del df

3.4.1 Función de Preprocesado

In [25]:
# Tokenizar y convertir texto a minúsculas
# Eliminar caracteres especiales
# Eliminar acentos y signos de puntuación
# Eliminar stopwords
# Convertir números a palabras
# Lematizar
# Eliminar las palabras más frecuentes
# Unir resultado en un texto limpio
# Aplicar funciones de preprocesamiento
# Especificar umbral de frecuencia para eliminar palabras frecuentes

def preprocess(text_data, freq_threshold=300):

    def tokenize_and_lowercase(text):
        tokens = word_tokenize(text.lower())
        return tokens

    def remove_special_characters(tokens):
        tokens = [word for word in tokens if word.isalnum()]
        return tokens

    def remove_accents_and_punctuation(tokens):
        cleaned_tokens = []
        for token in tokens:
          token = ''.join([char for char in unicodedata.normalize('NFKD', token) if not unicodedata.combining(char)])
          token = ''.join([char for char in token if char not in string.punctuation])
          cleaned_tokens.append(token)
        return cleaned_tokens

    def remove_stopwords(tokens):
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
        return tokens

    def numbers_to_words(tokens):
        tokens = [num2words(word) if word.isdigit() else word for word in tokens]
        return tokens

    def lemmatize_tokens(tokens):
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        return tokens

    def remove_common_words(tokens, freq_threshold):
        word_freq = Counter(tokens)
        tokens = [word for word in tokens if word_freq[word] <= freq_threshold]
        return tokens

    def join_tokens(tokens):
        text = ' '.join(tokens)
        return text

    text_data = text_data.apply(tokenize_and_lowercase)
    text_data = text_data.apply(remove_special_characters)
    text_data = text_data.apply(remove_accents_and_punctuation)
    text_data = text_data.apply(remove_stopwords)
    text_data = text_data.apply(numbers_to_words)
    text_data = text_data.apply(lemmatize_tokens)
    text_data = text_data.apply(lambda x: remove_common_words(x, freq_threshold))
    text_data = text_data.apply(join_tokens)

    return text_data

3.4.2 Aplicación del Preprocesado en df

In [26]:
#Llamar función de preprocesado

df_corpus['reviewText'] = preprocess(df_corpus['reviewText'], freq_threshold=300)
df_corpus.head(5)

,overall,reviewText
0,4,material arrived early excellent condition how...
1,4,really enjoying book worksheet make review goa...
2,1,taking class waste money called book book isi ...
3,3,book missing page important page could answer ...
4,5,used learnsmart officially say amazing study t...


3.5 Guardar el Dataframe preprocesado

In [27]:
# Guardar el DataFrame en Google Drive
df_corpus.to_csv('/content/drive/MyDrive/Bootcamp AI&ML KC/NLP/df_corpus.csv', index=False)

3.6 Conclusiones del preprocesado

a. Durante la limpieza de datos, se validaron valores NaN/Nulls (missing data), se encontraron 66 NaN en la columna: reviewText, debido a que, la proporción es de un 0.01436% respecto al total de reviews del df, se decidió eliminar esas filas.

b. En el Preprocesamiento se realizó lo siguiente:

-Tokenizar y convertir texto a minúsculas: Dividir el texto en palabras y al convertir el texto a minúsculas, se asegura la consistencia en el análisis, ya que las palabras en mayúsculas y minúsculas se tratan como iguales.

-Eliminar caracteres especiales: A menudo no aportan información significativa en el análisis de texto y al eliminarlas simplifican el texto.

-Eliminar acentos y signos de puntuación:  Asegurar que el texto esté libre de caracteres que no sean letras o números.

-Eliminar stopwords: No aportan información específica.

-Convertir números a palabras:  Para que sean más comprensibles en el análisis de texto.

-Lematizar: Agrupar palabras relacionadas.

-Eliminar las palabras más frecuentes: Reducir la dimensionalidad del texto y centrarse en las palabras más relevantes.

-Unir resultado en un texto limpio: Para proceder al modelado de lenguaje y análisis del texto.